In [1]:
import pandas as pd
import os
import tqdm
from sklearn.preprocessing import OneHotEncoder
import utils.featurecreations as featurecreations
import importlib

In [2]:
# dfs = []
# for file in os.listdir('data'):
#     try:
#         df = pd.read_excel(f'data/{file}')
#         dfs.append(df)
#     except:
#         pass

In [3]:
# data = pd.concat(dfs)

In [4]:
# data = data.reset_index(drop=True)

In [5]:
# data.to_csv('all_races_data.csv')

In [6]:
df = pd.read_feather('all_races_data.fth')

In [7]:
data = df

In [8]:
data['meetingId'] = data['meetingId'].astype(int).astype(str)
data['raceId'] = data['raceId'].astype(int).astype(str)
data['horseId'] = data['horseId'].astype(int).astype(str)

In [9]:
stats_cols = ['RPR', 'RPP', 'WP20R', 'WP20P', 'TP20R', 'TP20P', 'WPTP20R',
       'WPTP20P', 'RSFI', 'ScRank', 'SPrice', 'PFScore', 'PrSe', 'AHSP',
       ]

In [10]:
data.RunStyle.head()

0      op/mf
1      op/mf
2    no data
3      mf/bm
4      mf/bm
Name: RunStyle, dtype: object

In [11]:
data.RunStyle.value_counts()

RunStyle
mf         99023
bm         96145
op         67851
op/mf      63275
mf/bm      60258
l          29478
no data    26594
Name: count, dtype: int64

In [12]:
run_style_ohe = OneHotEncoder(sparse_output=False)
run_style_one_hot = run_style_ohe.fit_transform(data[['RunStyle']])
track_ohe = OneHotEncoder(sparse_output=False)
track_one_hot = track_ohe.fit_transform(data[['track']])

In [13]:
data["run_style_one_hot"] = run_style_one_hot.tolist()
data["tracks_one_hot"] = track_one_hot.tolist()

In [14]:
len(data["tracks_one_hot"].iloc[0])

397

In [15]:
horse_form = data.groupby(['horseId'], sort=False, as_index=False)

In [16]:
importlib.reload(featurecreations)

<module 'utils.featurecreations' from 'c:\\Users\\Nick\\Documents\\GitHub\\BetfairSpringCarnival\\src\\utils\\featurecreations.py'>

In [17]:
data = featurecreations.generate_prev_race(data, horse_form)

In [18]:
data['prev_race'].iloc[-1]

'962316'

In [19]:
data['dog_id'] = data['horseId']
data['box'] = data['barrier']
data['bfSP'] = data['priceBF']
data['margin'] = data['MargFin']
data['StartPrice'] = data['priceBF']
data['place'] = data['position']
data['runtime'] = data['ITimeToFin']
data['date'] = pd.to_datetime(data['meetingDate'], format='mixed', dayfirst=True ).dt.date
# data['date'] = pd.Timestamp(data['date'])
data['state'] = data['State']
data['dog_name'] = data['horse']

In [20]:
# data.barrier?
data.columns
data['State']

0          NZ
1          NZ
2          NZ
3          NZ
4          NZ
         ... 
530176    NSW
530177    NSW
530178    NSW
530179    NSW
530180    NSW
Name: State, Length: 530181, dtype: object

In [21]:
form_only_stats = data[stats_cols].fillna(-1.0)
stats = pd.Series(form_only_stats.values.tolist())

In [22]:
# data = data.drop(columns=stats_cols)
data['stats'] = stats

In [23]:
data = data.query('barrier > 0 and barrier <=20')
data.barrier.value_counts()

barrier
2     59000
1     58993
3     58918
4     58369
5     56513
6     52878
7     47387
8     40527
9     32733
10    25185
11    17305
12    11687
13     5737
14     3425
15      863
16      476
17       94
18       63
19       12
20        9
Name: count, dtype: int64

In [24]:
data['box'] = data['barrier'] -1

In [25]:
data['runtime'] = data['ITimeToFin']
data['race_grade'] = data['class']

In [26]:
data['stats_cols'] = str([stats_cols])
data['stats_cols'] = data['stats_cols'].astype('category')

In [27]:
data.reset_index().to_feather('horsey_racing_v1.fth')

In [28]:
data.columns

Index(['meetingId', 'raceId', 'horseId', 'meetingDate', 'track', 'trackAbbrev',
       'State', 'MPC', 'railPosition', 'TABMeeting',
       ...
       'bfSP', 'StartPrice', 'place', 'runtime', 'date', 'state', 'dog_name',
       'stats', 'race_grade', 'stats_cols'],
      dtype='object', length=108)

In [29]:
x = data.groupby('track')['barrier'].max()

In [30]:
data.barrier.value_counts()

barrier
2     59000
1     58993
3     58918
4     58369
5     56513
6     52878
7     47387
8     40527
9     32733
10    25185
11    17305
12    11687
13     5737
14     3425
15      863
16      476
17       94
18       63
19       12
20        9
Name: count, dtype: int64